In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("E:/NLP/Project/data/train.csv")

In [3]:
test_data = pd.read_csv("E:/NLP/Project/data/test.csv")

In [4]:
test_data.head(5)

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...


In [ ]:
'''To protect users from being exposed to offensive language on online forums or social media sites,
companies have started flagging comments and blocking users who are found guilty of using unpleasant language.
Several Machine Learning models have been developed and deployed to filter out the unruly language and protect 
internet users from becoming victims of online harassment and cyberbullying.'''

In [ ]:
#“To build a multi-headed model that’s capable of detecting different types of toxicity like threats, obscenity, insults, and identity-based hate.”

In [10]:
#Step 1: Checking for missing values in train data.
train_data =train_data[['id','comment_text']]
null_columns=train_data.columns[train_data.isnull().any()]
train_data[null_columns].isnull().sum()

Series([], dtype: float64)

In [11]:
#checking missing values in test data
null_columns=test_data.columns[test_data.isnull().any()]
test_data[null_columns].isnull().sum()

Series([], dtype: float64)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   id            int64 
 1   comment_text  object
dtypes: int64(1), object(1)
memory usage: 27.5+ MB


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            97320 non-null  int64 
 1   comment_text  97320 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [14]:
train_data.dropna()

,id,comment_text
0,59848,"This is so cool. It's like, 'would you want yo..."
1,59849,Thank you!! This would make my life a lot less...
2,59852,This is such an urgent design problem; kudos t...
3,59855,Is this something I'll be able to install on m...
4,59856,haha you guys are a bunch of losers.
...,...,...
1804869,6333967,"Maybe the tax on ""things"" would be collected w..."
1804870,6333969,What do you call people who STILL think the di...
1804871,6333982,"thank you ,,,right or wrong,,, i am following ..."
1804872,6334009,Anyone who is quoted as having the following e...


In [15]:
test_data.dropna()

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...
...,...,...
97315,7194635,He should lose his job for promoting mis-infor...
97316,7194636,"""Thinning project is meant to lower fire dange..."
97317,7194637,I hope you millennials are happy that you put ...
97318,7194638,I'm thinking Kellyanne Conway (a.k.a. The Trum...


In [16]:
#combined both the data sets 
combined_data = train_data.append(test_data)

In [17]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1902194 entries, 0 to 97319
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   id            int64 
 1   comment_text  object
dtypes: int64(1), object(1)
memory usage: 43.5+ MB


In [20]:
combined_data.comment_text[0]

0    This is so cool. It's like, 'would you want yo...
0    [ Integrity means that you pay your debts.]\n\...
Name: comment_text, dtype: object

In [22]:
combined_data = combined_data[['comment_text']]

In [24]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1902194 entries, 0 to 97319
Data columns (total 1 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   comment_text  object
dtypes: object(1)
memory usage: 109.0+ MB


In [67]:
df=combined_data.sample(n = 50000, random_state = 2)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1775687 to 1268339
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment_text  50000 non-null  object
dtypes: object(1)
memory usage: 781.2+ KB


In [29]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import p

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\akash\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https', 'did', 'and','www', 'just', 'com','do','to','s'])

In [71]:
# Convert to list
data = df.comment_text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub('[,\.!?]', "", sent) for sent in data]

pprint(data[:3])

['Perhaps ISIS should just sit back and watch angry white Gunmericans do their '
 'bidding for them',
 'Most people have smoked it and most people have moved on',
 '"Both theology and science seek to reveal the truth" They do Religion claims '
 'to already know "truth" offering absolutely no new information and no '
 'supporting evidence for claims it makes yet demanding unquestioned '
 'compliance with its ideas Science otoh tries to disprove claims that are '
 "made using evidence as it's basis for knowledge and changes it's mind when "
 'better information is found This is known as "learning" Try doing this with '
 "religion and see how far you get If religion was the basis for truth we'd "
 'still be living in the Dark Ages']


In [76]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['perhaps', 'isis', 'should', 'just', 'sit', 'back', 'and', 'watch', 'angry', 'white', 'gunmericans', 'do', 'their', 'bidding', 'for', 'them']]


In [77]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example   
print(trigram_mod[bigram_mod[data_words[0]]])

['perhaps', 'isis', 'should', 'just', 'sit', 'back', 'and', 'watch', 'angry', 'white', 'gunmericans', 'do', 'their', 'bidding', 'for', 'them']


In [78]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [119]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#['NOUN', 'ADJ', 'VERB', 'ADV']

print(data_lemmatized[:1])

[['perhaps', 'isis', 'sit', 'back', 'watch', 'angry', 'white', 'gunmerican', 'bid']]


In [120]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [121]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [122]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.016*"public" + 0.013*"family" + 0.012*"city" + 0.009*"play" + '
  '0.009*"house" + 0.009*"service" + 0.009*"kid" + 0.009*"report" + '
  '0.009*"home" + 0.008*"level"'),
 (1,
  '0.018*"get" + 0.016*"trump" + 0.015*"go" + 0.014*"make" + 0.013*"say" + '
  '0.012*"know" + 0.011*"think" + 0.011*"time" + 0.010*"take" + 0.010*"well"'),
 (2,
  '0.023*"pay" + 0.021*"money" + 0.015*"canadian" + 0.014*"job" + '
  '0.012*"business" + 0.011*"high" + 0.011*"cost" + 0.010*"less" + '
  '0.010*"base" + 0.010*"oil"'),
 (3,
  '0.023*"people" + 0.014*"country" + 0.011*"support" + 0.011*"american" + '
  '0.010*"world" + 0.010*"great" + 0.010*"system" + 0.007*"become" + '
  '0.007*"obama" + 0.007*"power"'),
 (4,
  '0.021*"year" + 0.015*"even" + 0.015*"work" + 0.014*"government" + '
  '0.010*"fact" + 0.010*"live" + 0.010*"canada" + 0.009*"first" + 0.008*"try" '
  '+ 0.008*"still"')]


In [123]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.683105096141983

Coherence Score:  0.491240715903988


In [124]:
#tuning
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [125]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.403649  0.003773       1        1  41.047312
4     -0.084434 -0.381363       2        1  20.623691
3     -0.035275  0.189581       3        1  16.592051
2     -0.152268  0.110717       4        1  12.236971
0     -0.131671  0.077292       5        1   9.499975, topic_info=               Term         Freq        Total Category  logprob  loglift
10           people  8070.000000  8070.000000  Default  30.0000  30.0000
337            year  5210.000000  5210.000000  Default  29.0000  29.0000
24              get  8608.000000  8608.000000  Default  28.0000  28.0000
895             pay  3307.000000  3307.000000  Default  27.0000  27.0000
109           trump  7771.000000  7771.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
2411           http   556.786881   557.614740   Topic5  -5.3099   2.3524
1195           open   556.526154   557.356677   Topic5  -5.3104   2.3524
823            road   542.190731   543.017507   Topic5  -5.3365   2.3524
146   investigation   528.133032   528.963530   Topic5  -5.3628   2.3523
949         involve   583.167893   680.906437   Topic5  -5.2636   2.1989

[192 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
864       4  0.998638       add
1417      3  0.999505    alaska
420       1  0.999875      also
937       3  0.999077   america
52        3  0.999494  american
...     ...       ...       ...
1408      4  0.999638      word
112       2  0.999735      work
660       3  0.999560     world
458       2  0.999252     wrong
337       2  0.999906      year

[171 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])

In [ ]:
###Topics:
#1. Trump
#2. Canada Government
#3.USA
#4.Jobs 
#5.Family


In [ ]:
#####ignore the code below

In [92]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df['comment_text'].values.astype('U'))

In [93]:
doc_term_matrix

<50000x31553 sparse matrix of type '<class 'numpy.int64'>'
	with 1068985 stored elements in Compressed Sparse Row format>

In [94]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [96]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

reparations
disordered
admonition
deceive
lane
emerging
alones
mathematics
pricing
client


In [97]:
first_topic = LDA.components_[0]

In [98]:
first_topic

array([41.44729296, 58.43563322,  2.19997328, ...,  0.20000066,
        0.20000066,  3.19877193])

In [99]:
top_topic_words = first_topic.argsort()[-10:]

In [100]:
top_topic_words

array([16763, 22428, 15949, 28450, 24894,  8917,  5466, 15623, 16630,
       20795], dtype=int64)

In [101]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

live
public
know
time
school
don
city
just
like
people


In [102]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['live', 'public', 'know', 'time', 'school', 'don', 'city', 'just', 'like', 'people']


Top 10 words for topic #1:
['com', 'oil', 'party', 'people', 'state', 'don', 'tax', 'like', 'just', 'trump']


Top 10 words for topic #2:
['country', 'white', 'news', 'don', 'right', 'did', 'just', 'like', 'people', 'trump']


Top 10 words for topic #3:
['women', 'god', 'church', 'know', 'good', 'think', 'just', 'like', 'don', 'people']


Top 10 words for topic #4:
['like', 'year', 'just', 'pay', 'money', 'years', 'tax', 'canada', 'people', 'government']




In [26]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(1804874, 5)

In [27]:
train_data['Topic'] = topic_values.argmax(axis=1)

In [28]:
train_data.head()

,comment_text,id,Topic
0,"This is so cool. It's like, 'would you want yo...",59848,1
1,Thank you!! This would make my life a lot less...,59849,1
2,This is such an urgent design problem; kudos t...,59852,1
3,Is this something I'll be able to install on m...,59855,1
4,haha you guys are a bunch of losers.,59856,1


In [29]:
unique(train_data.Topic)

NameError: name 'unique' is not defined

In [ ]:
#https://stackabuse.com/python-for-nlp-topic-modeling/#:~:text=Topic%20modeling%20is%20an%20unsupervised,clusters%20based%20on%20similar%20characteristics.